In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obstructive_sleep_apnea"
cohort = "GSE75097"

# Input paths
in_trait_dir = "../../input/GEO/Obstructive_sleep_apnea"
in_cohort_dir = "../../input/GEO/Obstructive_sleep_apnea/GSE75097"

# Output paths
out_data_file = "../../output/preprocess/Obstructive_sleep_apnea/GSE75097.csv"
out_gene_data_file = "../../output/preprocess/Obstructive_sleep_apnea/gene_data/GSE75097.csv"
out_clinical_data_file = "../../output/preprocess/Obstructive_sleep_apnea/clinical_data/GSE75097.csv"
json_path = "../../output/preprocess/Obstructive_sleep_apnea/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray gene expression profiles of peripheral blood mononuclear cells (PBMC) in patients with  obstructive sleep apnea and primary snoring"
!Series_summary	"Therefore, we extended our investigation into OSA patients with long-term continuous positive airway pressure (CPAP) treatment, hypertension, or excessive daytime sleepiness (EDS) by analyzing whole-genome gene expression profiles of PBMC in three comparisons: (1) treatment-naïve moderate to very severe OSA patients versus subjects with primary snoring; (2) moderate to very severe OSA patients with hypertension or EDS versus those without hypertension or EDS, respectively; (3) treatment-naïve very severe OSA patients versus those receiving at least one year of adequate CPAP treatment."
!Series_overall_design	"We analyzed whole-genome gene expression profiles of peripheral blood mononuclear cells from 48 patients with sleep-disordered breathing stratified into four groups: primary snoring (

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the series title and summary, this dataset contains microarray gene expression profiles
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Determine rows for each clinical feature
# From the sample characteristics dictionary, we can identify:
# Row 1: apnea hypopnea index (AHI) - this is a key clinical measure for OSA severity
trait_row = 1  # AHI values indicate OSA severity
age_row = 3    # Age information is available
gender_row = 2 # Gender information is available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert apnea hypopnea index (AHI) to a binary trait: OSA or not
    AHI < 5 is considered normal/primary snoring
    AHI ≥ 5 is considered OSA
    """
    if value is None:
        return None
    # Extract the value after the colon
    parts = str(value).split(': ')
    if len(parts) != 2:
        return None
    
    try:
        ahi = float(parts[1].strip())
        # AHI < 5 is considered normal/primary snoring (0)
        # AHI ≥ 5 is considered OSA (1)
        return 1 if ahi >= 5 else 0
    except:
        return None

def convert_age(value):
    """Convert age string to a numeric value."""
    if value is None:
        return None
    # Extract the value after the colon
    parts = str(value).split(': ')
    if len(parts) != 2:
        return None
    
    try:
        age = int(parts[1].strip())
        return age
    except:
        return None

def convert_gender(value):
    """Convert gender string to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    # Extract the value after the colon
    parts = str(value).split(': ')
    if len(parts) != 2:
        return None
    
    gender = parts[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

# 3. Save Metadata - Initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # For the geo_select_clinical_features function, we need to create a proper DataFrame
    # that's structured with row indices corresponding to the rows in the sample characteristics

    # Define the sample characteristics from the previous step
    sample_chars = {
        0: ['cell type: PBMC'], 
        1: ['apnea hyponea index: 22.7', 'apnea hyponea index: 32.6', 'apnea hyponea index: 56.5', 
            'apnea hyponea index: 46.9', 'apnea hyponea index: 31.1', 'apnea hyponea index: 4.5', 
            'apnea hyponea index: 26.7', 'apnea hyponea index: 56.4', 'apnea hyponea index: 22.6', 
            'apnea hyponea index: 33.4', 'apnea hyponea index: 98.6', 'apnea hyponea index: 73.5', 
            'apnea hyponea index: 63.3', 'apnea hyponea index: 44.1', 'apnea hyponea index: 50.2', 
            'apnea hyponea index: 43.8', 'apnea hyponea index: 63.4', 'apnea hyponea index: 79.2', 
            'apnea hyponea index: 42.1', 'apnea hyponea index: 24.3', 'apnea hyponea index: 2.4', 
            'apnea hyponea index: 59.9', 'apnea hyponea index: 73.2', 'apnea hyponea index: 64.9', 
            'apnea hyponea index: 33.2', 'apnea hyponea index: 45.6', 'apnea hyponea index: 4.3', 
            'apnea hyponea index: 85.1', 'apnea hyponea index: 28.4', 'apnea hyponea index: 86.5'], 
        2: ['Sex: male', 'Sex: female'], 
        3: ['age: 54', 'age: 31', 'age: 44', 'age: 60', 'age: 21', 'age: 50', 'age: 52', 
            'age: 58', 'age: 42', 'age: 34', 'age: 37', 'age: 59', 'age: 27', 'age: 57', 
            'age: 68', 'age: 53', 'age: 36', 'age: 38', 'age: 43', 'age: 46', 'age: 49', 
            'age: 61', 'age: 45', 'age: 35', 'age: 47'], 
        4: ['hypertension: 1', 'hypertension: 0'], 
        5: ['eds: 0', 'eds: 1']
    }
    
    # Create a DataFrame that's compatible with geo_select_clinical_features
    # The function likely expects a DataFrame with row indices matching the sample characteristics
    clinical_data = pd.DataFrame(index=range(max(sample_chars.keys()) + 1))
    
    # Add an empty column to make it a proper DataFrame
    clinical_data['values'] = [sample_chars.get(i, []) for i in range(len(clinical_data))]
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'values': [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Obstructive_sleep_apnea/clinical_data/GSE75097.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')

Gene data dimensions: 34677 genes × 48 samples


### Step 4: Gene Identifier Review

In [5]:
# The identifiers appear to be a mix of gene symbols (like A1BG, A2M) and some less standard identifiers.
# Most of them look like recognized gene symbols, with a few exceptions.
# Given that most seem to be proper gene symbols, we can consider this as not requiring mapping.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist()}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data using the correct row indices and conversion functions from step 2
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get background information and clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Use the correct values and functions identified in step 2
trait_row = 1  # apnea hypopnea index
age_row = 3    # age information
gender_row = 2 # gender information

def convert_trait(value):
    """
    Convert apnea hypopnea index (AHI) to a binary trait: OSA or not
    AHI < 5 is considered normal/primary snoring
    AHI ≥ 5 is considered OSA
    """
    if value is None:
        return None
    # Extract the value after the colon
    parts = str(value).split(': ')
    if len(parts) != 2:
        return None
    
    try:
        ahi = float(parts[1].strip())
        # AHI < 5 is considered normal/primary snoring (0)
        # AHI ≥ 5 is considered OSA (1)
        return 1 if ahi >= 5 else 0
    except:
        return None

def convert_age(value):
    """Convert age string to a numeric value."""
    if value is None:
        return None
    # Extract the value after the colon
    parts = str(value).split(': ')
    if len(parts) != 2:
        return None
    
    try:
        age = int(parts[1].strip())
        return age
    except:
        return None

def convert_gender(value):
    """Convert gender string to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    # Extract the value after the colon
    parts = str(value).split(': ')
    if len(parts) != 2:
        return None
    
    gender = parts[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

# Extract clinical features with correct row indices and conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Debug: Show preview of clinical data
print("Preview of clinical data:")
print(preview_df(selected_clinical_df))

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available in step 2
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression profiles from patients with obstructive sleep apnea and primary snoring."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...


Gene data shape after normalization: (20741, 48)
First 5 normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1']


Normalized gene data saved to ../../output/preprocess/Obstructive_sleep_apnea/gene_data/GSE75097.csv
Preview of clinical data:
{'GSM1942590': [1.0, 54.0, 1.0], 'GSM1942591': [1.0, 31.0, 1.0], 'GSM1942592': [1.0, 44.0, 1.0], 'GSM1942593': [1.0, 60.0, 1.0], 'GSM1942594': [1.0, 21.0, 1.0], 'GSM1942595': [0.0, 50.0, 1.0], 'GSM1942596': [1.0, 52.0, 0.0], 'GSM1942597': [1.0, 58.0, 1.0], 'GSM1942598': [1.0, 42.0, 0.0], 'GSM1942599': [1.0, 34.0, 1.0], 'GSM1942600': [1.0, 58.0, 1.0], 'GSM1942601': [1.0, 37.0, 1.0], 'GSM1942602': [1.0, 60.0, 0.0], 'GSM1942603': [1.0, 59.0, 1.0], 'GSM1942604': [1.0, 27.0, 1.0], 'GSM1942605': [1.0, 57.0, 1.0], 'GSM1942606': [1.0, 68.0, 1.0], 'GSM1942607': [1.0, 53.0, 1.0], 'GSM1942608': [1.0, 58.0, 1.0], 'GSM1942609': [1.0, 52.0, 0.0], 'GSM1942610': [0.0, 36.0, 1.0], 'GSM1942611': [1.0, 38.0, 1.0], 'GSM1942612': [1.0, 50.0, 1.0], 'GSM1942613': [1.0, 44.0, 1.0], 'GSM1942614': [1.0, 58.0, 1.0], 'GSM1942615': [1.0, 54.0, 1.0], 'GSM1942616': [0.0, 43.0, 1.0], 'GSM1942

Data shape after handling missing values: (48, 20744)
For the feature 'Obstructive_sleep_apnea', the least common label is '0.0' with 5 occurrences. This represents 10.42% of the dataset.
The distribution of the feature 'Obstructive_sleep_apnea' in this dataset is fine.

Quartiles for 'Age':
  25%: 42.0
  50% (Median): 50.0
  75%: 58.0
Min: 21.0
Max: 68.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 9 occurrences. This represents 18.75% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Obstructive_sleep_apnea/GSE75097.csv
